In [1]:
# 토큰 정보로드를 위한 라이브러리
# 설치: pip install python-dotenv
from dotenv import load_dotenv

# 토큰 정보로드
load_dotenv()

True

## ChatMessageHistory

대부분의 메모리 모듈을 뒷받침하는 핵심 유틸리티 클래스 중 하나는 ChatMessageHistory 클래스입니다.

이 클래스는 초경량 래퍼로, `HumanMessage`, `AIMessage`를 저장한 다음 모두 가져오는 편리한 메서드를 제공합니다.

체인 외부에서 메모리를 관리하는 경우 이 클래스를 직접 사용할 수 있습니다.


In [5]:
from langchain.memory import ChatMessageHistory

history = ChatMessageHistory()

history.add_user_message("안녕?!")
history.add_ai_message("반갑습니다. 무엇을 도와 드릴까요?")

In [6]:
# 출력
print(history)

Human: 안녕?!
AI: 반갑습니다. 무엇을 도와 드릴까요?


In [7]:
history.add_user_message("너는 누구니?")
history.add_ai_message("제 이름은 랭체인입니다. 무엇을 도와 드릴까요?")

In [8]:
# 출력
print(history)

Human: 안녕?!
AI: 반갑습니다. 무엇을 도와 드릴까요?
Human: 너는 누구니?
AI: 제 이름은 랭체인입니다. 무엇을 도와 드릴까요?


## 대화 내용을 기억하는 Chain 생성


In [9]:
from operator import itemgetter

from langchain_openai import ChatOpenAI
from langchain.memory import ConversationBufferMemory
from langchain.schema.runnable import (
    RunnablePassthrough,
    RunnableLambda,
    RunnableParallel,
)
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder

In [10]:
# 대화 히스토리 저장
chat_history = ChatMessageHistory()

# 대화내용 전달을 위한 메모리
memory = ConversationBufferMemory(
    return_messages=True,
    chat_memory=chat_history,
    memory_key="chat_memory"
)

In [11]:
# chat_memory 키에 저장되어 있는 대화 목록을 확인할 수 있습니다.
memory.load_memory_variables({})

{'chat_memory': []}

In [9]:
# 대화내용을 Chain 안에서 가져오기 위한 RunnableLambda 적용
RunnableLambda(memory.load_memory_variables).invoke({})

{'chat_memory': []}

In [10]:
# 대화내용을 Chain 안에서 가져오기 위한 RunnableLambda 적용
(RunnableLambda(memory.load_memory_variables) | itemgetter("chat_memory")).invoke({})

[]

In [12]:
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful chatbot"),
        MessagesPlaceholder(variable_name="chat_memory"),
        ("human", "{question}"),
    ]
)

model = ChatOpenAI(model="gpt-3.5-turbo")

In [13]:
chain = (
    RunnableParallel(
        chat_memory=RunnableLambda(memory.load_memory_variables)
        | itemgetter("chat_memory"),
        question=RunnablePassthrough(),
    )
    | prompt
    | model
)

In [14]:
q = "안녕하세요? 제 이름은 테디에요"
response = chain.invoke(q)
memory.save_context({"user": q}, {"ai": response.content})
print(f"HUMAN: {q}")
print(f"AI: {response.content}")

HUMAN: 안녕하세요? 제 이름은 테디에요
AI: 안녕하세요, 테디님! 무엇을 도와드릴까요?


In [15]:
q = "제 이름이 뭔지 기억하세요?"
response = chain.invoke(q)
memory.save_context({"user": q}, {"ai": response.content})
print(f"HUMAN: {q}")
print(f"AI: {response.content}")

HUMAN: 제 이름이 뭔지 기억하세요?
AI: 네, 테디님의 이름을 기억하고 있습니다! 어떻게 도와드릴까요?


In [16]:
# 대화내용을 모두 출력하여 확인합니다.
memory.load_memory_variables({})["chat_memory"]

[HumanMessage(content='안녕하세요? 제 이름은 테디에요'),
 AIMessage(content='안녕하세요, 테디님! 무엇을 도와드릴까요?'),
 HumanMessage(content='제 이름이 뭔지 기억하세요?'),
 AIMessage(content='네, 테디님의 이름을 기억하고 있습니다! 어떻게 도와드릴까요?')]

In [ ]:
# 대화내용을 비웁니다.
chat_history.clear()

In [ ]:
# 대화내용을 모두 출력하여 확인합니다.
memory.load_memory_variables({})["chat_memory"]

In [ ]:
q = "제 이름이 뭔지 기억하세요?"
response = chain.invoke(q)
memory.save_context({"user": q}, {"ai": response.content})
print(f"HUMAN: {q}")
print(f"AI: {response.content}")

In [ ]:
chat_history.messages[-2:]